In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
M = 300 #number of random walkers
R = np.zeros((2*M,3))
for m in range(2*M):
    for d in range(3):
        R[m,d] = np.random.uniform(-0.5, 0.5)

In [ ]:
delta_t = 0.03 
N = 40000 #Todo: 10000 are equilibration
kappa = 2 
beta = 1/2
alpha = 0.15 #Todo: Use the correct alpha

In [ ]:


def DMC_algorithm(initial_state, num_trials, s, alpha, beta, kappa, use_FP = False, delta_t=1, disable_tqdm = False):
    sequence_states = [initial_state]
    for _ in tqdm(range(num_trials), disable=disable_tqdm):
        next_sequence = DMC_step(sequence_states[-1], alpha, beta, s, kappa, delta_t=delta_t, use_FP=use_FP)
        sequence_states.append(next_sequence)
    return np.array(sequence_states)
   
def DMC_step(current_state, alpha, beta, s, kappa, delta_t = 1, use_FP=False):
    trial_configuration = current_state.copy()
    new_state = np.zeros(current_state.shape)
    for i in range(M):
        which_particle = np.random.choice([0,M]) # Todo: Is this chosen for each of the M seperately?
        if use_FP:
            trial_configuration[i+which_particle] += Fokker_planck(current_state[i+which_particle], current_state[i+M-which_particle], alpha, kappa, beta)*delta_t/2 #Todo: Is this correct?
        trial_configuration[i+which_particle] += np.random.uniform(low=-s/2, high=s/2, size=3)*np.sqrt(delta_t)
        p = (psi_T(trial_configuration[i], trial_configuration[i+M], alpha, kappa, beta)/psi_T(current_state[i], current_state[i+M], alpha, kappa, beta))**2
        if use_FP:
            g1 = greens_function(trial_configuration[i], current_state[i], alpha, beta, kappa, delta_t)*greens_function(trial_configuration[i+M], current_state[i+M], alpha, beta, kappa, delta_t)
            g2 = greens_function(trial_configuration[i], current_state[i], alpha, beta, kappa, delta_t)*greens_function(trial_configuration[i+M], current_state[i+M], alpha, beta, kappa, delta_t)
            p *= g1/g2                    
        trial_acceptance_prob = np.min([1, p])
        r_accept = np.random.rand()
        if r_accept < trial_acceptance_prob:
            new_state[i] = trial_configuration[i]
            new_state[i+M] = trial_configuration[i+M]
        else:
            new_state[i] = current_state[i]
            new_state[i+M] = current_state[i+M]
    return np.array(new_state)